In [14]:
import sqlite3 
import pandas as pd
import numpy as np
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import time
import requests
import re
from tqdm import tqdm

In [15]:
day = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}

In [38]:
webtoon_list = []
writers_list = []
painters_list = []
novel_origin_authors_list = []

for day_code in tqdm(day):
    url = f'https://gateway-kw.kakao.com/section/v2/timetables/days?placement=timetable_{day_code}'

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # pprint(data)
        for block in data.get("data", []):
            weekday = block.get("title")
            for group in block.get("cardGroups", []):
                for card in group.get("cards", []):
                    content = card.get("content", {})
                    additional = card.get("additional", {})
                    authors = content.get("authors", [])
                    badges = content.get("badges", [])
                    genre_filters = card.get("genreFilters", [])
                    adult_filters = card.get("additional", {})

                    author_names = ", ".join(a.get("name", "") for a in authors)
                    badge_titles = [b.get("title", "") for b in badges]

                    row = {
                        "웹툰ID": content.get("id"),
                        "제목": content.get("title"),
                        "seoId": content.get("seoId"),
                        "url": f'https://webtoon.kakao.com/content/{content.get("seoId")}/{content.get("id")}',
                        "thumbnailurl" : content.get("featuredCharacterImageA")+'.png',
                        "장르필터": ", ".join(genre_filters),
                        "카피문구": content.get("catchphraseTwoLines"),
                        "작가들": author_names,
                        "뱃지목록": ", ".join(badge_titles),
                        "UP여부": any(t == "up" for t in badge_titles),
                        "성인여부": adult_filters.get("adult"),
                        "요일": weekday,
                    }
                    webtoon_list.append(row)

    else:
        print("에러:", response.status_code)

print(f"웹툰 수집: {len(webtoon_list)}개")
# print(f"글 작가 정보: {len(writers_list)}개")

100%|██████████| 7/7 [00:00<00:00,  9.85it/s]

웹툰 수집: 1038개


In [39]:
df = pd.DataFrame(webtoon_list)
df.head()

,웹툰ID,제목,seoId,url,thumbnailurl,장르필터,카피문구,작가들,뱃지목록,UP여부,성인여부,요일
0,2589,대사형 선유,대사형-선유,https://webtoon.kakao.com/content/대사형-선유/2589,https://kr-a.kakaopagecdn.com/P/C/2589/c1/2x/8...,"all, ACTION_WUXIA",무영문의 대사형 선유.\n그런 그의 우직한 강호이야기.,"노경찬, 박창환, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,월
1,2043,무지개다리 파수꾼,무지개다리-파수꾼,https://webtoon.kakao.com/content/무지개다리-파수꾼/2043,https://kr-a.kakaopagecdn.com/P/C/2043/c1/2x/6...,"all, FANTASY_DRAMA","돈과 명예만을 좇던 유명 수의사,\n동물의 소리를 듣게 되다!","이서, 이서, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,월
2,2385,4000년 만에 귀환한 대마도사,4000년-만에-귀환한-대마도사,https://webtoon.kakao.com/content/4000년-만에-귀환한...,https://kr-a.kakaopagecdn.com/P/C/2385/c1/2x/5...,"all, SCHOOL_ACTION_FANTASY",4000년의 시간을 넘어 귀환한\n대마도사의 화려한 액션이 시작된다!,"따개비, 김덕용(REDICE STUDIO), 낙하산, 콘텐츠랩블루",WAIT_FOR_FREE,False,False,월
3,2473,이번 생은 가주가 되겠습니다,이번-생은-가주가-되겠습니다,https://webtoon.kakao.com/content/이번-생은-가주가-되겠...,https://kr-a.kakaopagecdn.com/P/C/2473/c1/2x/6...,"all, ROMANCE_FANTASY","환생에 회귀까지, 인생 3회차\n피렌티아의 가주되기 프로젝트!","ANTSTUDIO, 몬(ANTSTUDIO), 김로아, 디앤씨웹툰",WAIT_FOR_FREE,False,False,월
4,3345,교룡의 주인,교룡의-주인,https://webtoon.kakao.com/content/교룡의-주인/3345,https://kr-a.kakaopagecdn.com/P/C/3345/c1/2x/4...,"all, ROMANCE_FANTASY",내 교룡이 된 것을 \n후회하지 않게 해주마.,"박래모, 박래모, 은소로, 테라핀",WAIT_FOR_FREE,False,False,월


In [40]:
cols_keep = ["웹툰ID", "제목", "seoId", "url", "thumbnailurl", "장르필터",
             "카피문구", "작가들", "뱃지목록", "UP여부", "성인여부"]

df = (
    df.groupby(cols_keep, as_index=False)["요일"]
      .agg(lambda s: ", ".join(sorted(set(s))))
)

df.head()

,웹툰ID,제목,seoId,url,thumbnailurl,장르필터,카피문구,작가들,뱃지목록,UP여부,성인여부,요일
0,198,오무라이스 잼잼,오무라이스-잼잼,https://webtoon.kakao.com/content/오무라이스-잼잼/198,https://kr-a.kakaopagecdn.com/P/C/198/c1/2x/dc...,"all, COMIC_EVERYDAY_LIFE",다이어트 할 때 가장 위험한 만화\n추억과 음식과 이야기들,"조경규, 조경규, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,화
1,760,딩스뚱스,딩스뚱스,https://webtoon.kakao.com/content/딩스뚱스/760,https://kr-a.kakaopagecdn.com/P/C/760/c1/2x/29...,"all, COMIC_EVERYDAY_LIFE","토종 한국인 딩스와 뚱스,\n타지에서 맨땅에 헤딩하기!","딩스, 딩스, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,화
2,792,블랙 베히모스,블랙-베히모스,https://webtoon.kakao.com/content/블랙-베히모스/792,https://kr-a.kakaopagecdn.com/P/C/792/c1/2x/48...,"all, SCHOOL_ACTION_FANTASY",탈리스만이 되기 위한 시험에 \n도전하는 주인공의 힘겨운 전투!,"케이지콘, 케이지콘, 카카오웹툰 스튜디오","FREE_PUBLISHING, up",True,False,수
3,823,레드스톰 - 왕의 귀환,레드스톰---왕의-귀환,https://webtoon.kakao.com/content/레드스톰---왕의-귀환...,https://kr-a.kakaopagecdn.com/P/C/823/c1/2x/fc...,"all, ACTION_WUXIA","4년 만에 돌아온 그로우, 왕의 귀환\n사막을 넘어 다시 시작되는 전쟁","노경찬, 암현, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,토
4,1078,유부녀의 탄생,유부녀의-탄생,https://webtoon.kakao.com/content/유부녀의-탄생/1078,https://kr-a.kakaopagecdn.com/P/C/1078/c1/2x/0...,"all, COMIC_EVERYDAY_LIFE",누구도 알려주지 않은 결혼식 이후\n삶에 대한 리얼리티 카툰!,"김환타, 김환타, 카카오웹툰 스튜디오",FREE_PUBLISHING,False,False,월


In [41]:
API_URL = "https://gateway-kw.kakao.com/decorator/v2/decorator/contents/{id}/profile"

In [42]:
def fetch_detail(content_id: int) -> dict:
    url = API_URL.format(id=content_id)
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()["data"]

In [43]:
def parse_detail(detail: dict) -> dict:
    synopsis = detail.get("synopsis", "")

    raw_keywords = detail.get("seoKeywords", []) or []
    seo_keywords = [kw.lstrip("#").strip() for kw in raw_keywords]

    authors = detail.get("authors", []) or []
    writers, artists, originals = [], [], []

    for a in authors:
        name = (a.get("name") or "").strip()
        role = a.get("type")
        if not name:
            continue

        if role == "AUTHOR":
            writers.append(name)
        elif role == "ILLUSTRATOR":
            artists.append(name)
        elif role == "ORIGINAL_STORY":
            originals.append(name)

    return {
        "synopsis": synopsis,
        "seo_keywords": seo_keywords,
        "writers": writers,
        "artists": artists,
        "original_authors": originals,
    }

In [44]:
rows = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    webtoon_id = int(row["웹툰ID"])

    try:
        detail = fetch_detail(webtoon_id)
        parsed = parse_detail(detail)
    except Exception as e:
        print("failed:", webtoon_id, e)
        parsed = {
            "synopsis": "",
            "seo_keywords": [],
            "writers": [],
            "artists": [],
            "original_authors": [],
        }

    new_row = row.copy()
    new_row["시놉시스"] = parsed["synopsis"]
    new_row["키워드리스트"] = ", ".join(parsed["seo_keywords"])
    new_row["글작가"] = ", ".join(parsed["writers"])
    new_row["그림작가"] = ", ".join(parsed["artists"])
    new_row["원작가"] = ", ".join(parsed["original_authors"])
    rows.append(new_row)

df_enriched = pd.DataFrame(rows)

100%|██████████| 1026/1026 [00:58<00:00, 17.69it/s]


In [45]:
df_kakao = df_enriched.drop(["seoId", "뱃지목록", "작가들", "UP여부"], axis=1)
df_kakao.head()

,웹툰ID,제목,url,thumbnailurl,장르필터,카피문구,성인여부,요일,시놉시스,키워드리스트,글작가,그림작가,원작가
0,198,오무라이스 잼잼,https://webtoon.kakao.com/content/오무라이스-잼잼/198,https://kr-a.kakaopagecdn.com/P/C/198/c1/2x/dc...,"all, COMIC_EVERYDAY_LIFE",다이어트 할 때 가장 위험한 만화\n추억과 음식과 이야기들,False,화,'음식'과 '가족'의 이야기를 담은 에세이 만화.\n\n몇 개라도 먹을 수 있을 것...,"공감되는, 식욕을 자극하는, 코믹/일상, 지식정보",조경규,조경규,
1,760,딩스뚱스,https://webtoon.kakao.com/content/딩스뚱스/760,https://kr-a.kakaopagecdn.com/P/C/760/c1/2x/29...,"all, COMIC_EVERYDAY_LIFE","토종 한국인 딩스와 뚱스,\n타지에서 맨땅에 헤딩하기!",False,화,"토종 한국인 딩스와 뚱스, 타지에서 맨땅에 헤딩하기!","귀여운, 따뜻한, 코믹/일상, 에피소드물",딩스,딩스,
2,792,블랙 베히모스,https://webtoon.kakao.com/content/블랙-베히모스/792,https://kr-a.kakaopagecdn.com/P/C/792/c1/2x/48...,"all, SCHOOL_ACTION_FANTASY",탈리스만이 되기 위한 시험에 \n도전하는 주인공의 힘겨운 전투!,False,수,"태초에 세 명의 신이 있었다.\n땅의 베히모스, 바다의 레비야탄, 하늘의 즈가.\n...","몰입되는, 압도되는, 학원/판타지, 성장물",케이지콘,케이지콘,
3,823,레드스톰 - 왕의 귀환,https://webtoon.kakao.com/content/레드스톰---왕의-귀환...,https://kr-a.kakaopagecdn.com/P/C/823/c1/2x/fc...,"all, ACTION_WUXIA","4년 만에 돌아온 그로우, 왕의 귀환\n사막을 넘어 다시 시작되는 전쟁",False,토,피에 젖은 사막.\n붉은 사막을 용납할 수 없던 한 소년.\n\n소년은 사막의 무신...,"역동적인, 몰입되는, 액션/무협, 성공성장물",노경찬,암현,
4,1078,유부녀의 탄생,https://webtoon.kakao.com/content/유부녀의-탄생/1078,https://kr-a.kakaopagecdn.com/P/C/1078/c1/2x/0...,"all, COMIC_EVERYDAY_LIFE",누구도 알려주지 않은 결혼식 이후\n삶에 대한 리얼리티 카툰!,False,월,누구도 알려주지 않은 \n결혼식 이후 삶에 대한 리얼리티 카툰!,"공감되는, 귀여운, 코믹/일상, 에피소드물",김환타,김환타,


In [46]:
def merge_genre_into_keywords(row):
    # 1) 장르필터에서 all 제거
    raw = str(row["장르필터"])
    genres = [g.strip() for g in raw.split(",") if g.strip() and g.strip().lower() != "all"]

    # 2) 기존 키워드리스트 분해
    kw_raw = str(row["키워드리스트"])
    if kw_raw in ("nan", "None"):
        keywords = []
    else:
        keywords = [k.strip() for k in kw_raw.split(",") if k.strip()]

    # 3) 두 리스트 합치고 중복 제거
    merged = []
    for x in keywords + genres:
        if x not in merged:
            merged.append(x)

    return ", ".join(merged)

df_kakao["키워드리스트"] = df_kakao.apply(merge_genre_into_keywords, axis=1)
df_kakao["장르필터"] = df_kakao["장르필터"].str.replace(r"\ball\b,?\s*", "", regex=True).str.strip(", ")

In [47]:
df_kakao_final = df_kakao.drop(["장르필터"], axis=1)
df_kakao_final.to_csv("kakao_webtoons_final.csv", index=False)